# Chapter 9
In this chapter we look at:
* Classes and class relationships
* Syntax/code-idioms
* Exhaustive search
* Previous Chapters

In [chapter 1](chapter1.ipynb) (a closely related notebook see hyperlink) We look at optimization outputs that came from an optimization using spike waveform shape. 

In [chapter 2](chapter2.ipynb) (a closely related notebook see hyperlink) I demonstrate optimization using spike time statistics via the allen SDK Chapter 2

In [chapter 3](chapter3.ipynb) (a closely related notebook see hyperlink) We will take a closer at the neuroelectro data used to perform the fits in notebook1/Chapter 1. Specifically we will look at where optimized model behavior fits back onto the distribution of neuroelectro data.

In [chapter 5](chapter5.ipynb) (a closely related notebook see hyperlink) We will look at projections of Optimized cells onto a Druckman feature space, we will also look at extracting Allen SDK features from the optimized cells. Chapter 3


# Syntax and Idioms
## Data Transport Container
The main way of storing model attributes and moving non-parallel models tests around on different CPUs

Class Definition is in /neuronunit/neuronunit/optimisation/data_transport_container.py

Inputs and outputs of Optimization, are typically Data Transport Containers. 

dtc objects now contain a method, in order to quickly recover the model represented by a particular data transport container.
``` python
model = dtc.dtc_to_model()
```
DataTransport Container also includes a method judge tests, to judge any test suites that may be stored inside a model.

``` python
model = dtc.judge_tests()
```
..., and a method to convert dtc objects to genes:


``` python
model = dtc.dtc_to_gene()
```

## Test Suite Dictionary
An extension of pythons dictionary class. It is capable of calling a method ```optimize``` on itself.

* This type also has an attribute, use_rheobase in scores.

* Since not all experimental cell test suites, use rheobase to judge model fitness (Allen SDK does not). It's simpler to decide if rheobase calculations will contribute to Neuronunit scores at the users entrypoint to optimization.

 /neuronunit/neuronunit/optimisation/optimization_management.py

## Class OptMan
* An Optimization Management Class

Located at
 /neuronunit/neuronunit/optimisation/optimization_management.py

This object contains the method ```run_ga```, ```round_trip_test``` and ```run_grid```.
The object contains logic for switching between elephant and allen based measurements of waveform and firing. This object also contains logic for rheobase guided searches, or many firing waveform searches.

Once an optimization has been described using the Optmization management object a secondary object is constructed.

The ```OptMan``` object also performs tasks like creating new genes to replace models that had unstable parameters

# Some examples of using the OptMan class to optimize:


``` python
OM = OptMan(local_tests,backend=str('RAW'),boundary_dict=MODEL_PARAMS['RAW'],protocol={'elephant':True,'allen':False,'dm':False})#'tsr':spk_range})

grid_results = OM.run_simple_grid(5,free_params=['a','b','c'])
```

To simulate data, and internall check multi-objective high dimensional optimization:

``` python

OM = OptMan(use_test,protocol={'elephant':True,'allen':False,'dm':False})
results,converged,target,simulated_tests = OM.round_trip_test(use_test,
                                                              str('RAW'),\
                                                              MU=10,NGEN=10,stds = easy_standards)

```
To optimize against external neuro-electro data with an axaptive exponential backend:
``` python
ga_outad, DO = OM.run_ga(model_params.MODEL_PARAMS['ADEXP'],3, local_tests, free_params = model_params.MODEL_PARAMS['ADEXP'], MU = 10, backend = str('ADEXP'))
```
The run_ga method is now wrapped inside a ***Neuronunit Test Suite*** dictionary class. 

``` python
# test_dic an actual simple dictionary of NU Tests:
keys: test names, values test objects:

sim_tests = TSD(test_dic)

dtcpop = sim_tests.optimize(model_parameters.MODEL_PARAMS[backend], NGEN=50, \
                             backend=backend, MU=24, protocol={'allen': True, 'elephant': False,'tsr':results['dtc_pop'][0].tsr})

```



This Object constructs another object called  ```SciUnitOptimization``` that takes care of deap configuration. This file is departs from BluePyOpts base class, but it is similar to it.

The class sciunitoptimization largely calls methods in the files:
* neuronunit/optimisation/algorithms.py***
* neuronunit/optimisation/optimisations.py***

## These two files still largely follow BluePyOpt and DEAP conventions (expressed in DEAPS nsga2.py example file):

* ***algorithms.py***
* ***optimisations.py***

* With the differences: dask.bag.map is used to parallel map instead of scoop etc.

* nsga3 is used in the place of nsga2

* algorithms -> optimization_management.py ```def evaluate()``` 


This class (appropriated from BluePyOpt) is used to create new genes to replace models with unstable behavior.
```python
class WSListIndividual(list):
    """Individual consisting of list with weighted sum field"""
    def __init__(self, *args, **kwargs):
        """Constructor"""
        self.rheobase = None
        super(WSListIndividual, self).__init__(*args, **kwargs)

```



In a very simple example below. The OptMan class is instanced and used to perform simple optimization jobs including:
Exhaustive Search, running a genetic algorithm (nsga3), or round trip testing with simulated data


In [1]:
%%capture
from allensdk.ephys.extract_cell_features import extract_cell_features
import pickle
from neuronunit.optimisation.optimization_management import OptMan, TSD
from neuronunit.optimisation import model_parameters
from neuronunit.optimisation.model_parameters import MODEL_PARAMS
import pandas as pd
import os



In [2]:
import os
electro_path = str(os.getcwd())+'/../tests/russell_tests.p'

assert os.path.isfile(electro_path) == True
with open(electro_path,'rb') as f:
    (test_frame,obs_frame) = pickle.load(f)
local_tests = test_frame['Hippocampus CA1 pyramidal cell']
local_tests = TSD(local_tests)
local_tests.use_rheobase_score = True    


In [3]:
%%capture
free_params=['b','C','peak_v']
import copy
for f in free_params:
    hold_constant = copy.copy(MODEL_PARAMS['ADEXP'])
    hold_constant.pop(f,'None')

OM = OptMan(local_tests,backend=str('ADEXP'),\
            boundary_dict=MODEL_PARAMS['ADEXP'],\
            protocol={'elephant':True,'allen':False,'dm':False},\
            hc=hold_constant)#'tsr':spk_range})

ga_results = OM.optimize(free_params=['b','C','peak_v'],MU=3,NGEN=3)


True
['C', 'b', 'peak_v']
['C', 'b', 'peak_v']
['C', 'b', 'peak_v']
[<neuronunit.optimisation.data_transport_container.DataTC object at 0x1c2a9a4518>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x1c2a9a4940>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x1c2a9a4908>]
{'ADAPTATION_TIME_CONSTANT_tau_w': 100.0, 'ADAPTATION_VOLTAGE_COUPLING_a': 0.5, 'FIRING_THRESHOLD_v_spike': -30.0, 'MEMBRANE_RESISTANCE_R': 0.234375, 'MEMBRANE_TIME_SCALE_tau_m': 5.0, 'RHEOBASE_THRESHOLD_v_rh': -50.0, 'SHARPNESS_delta_T': 1.0, 'SPIKE_TRIGGERED_ADAPTATION_INCREMENT_b': 7.0, 'V_RESET': -51.0, 'V_REST': -70.0}
{'ADAPTATION_TIME_CONSTANT_tau_w': 100.0, 'ADAPTATION_VOLTAGE_COUPLING_a': 0.5, 'FIRING_THRESHOLD_v_spike': -30.0, 'MEMBRANE_RESISTANCE_R': 0.234375, 'MEMBRANE_TIME_SCALE_tau_m': 5.0, 'RHEOBASE_THRESHOLD_v_rh': -50.0, 'SHARPNESS_delta_T': 1.0, 'SPIKE_TRIGGERED_ADAPTATION_INCREMENT_b': 7.0, 'V_RESET': -51.0, 'V_REST': -70.0}
{'ADAPTATION_TIME_CONSTANT_tau_w'

In [ ]:
# res = OM.round_trip_test(local_tests,str('RAW'),MU=1,NGEN=1)#,stds = easy_standards)
ga_results[0]['pf'][0].dtc.scores
ga_results[0]['pf'][0].dtc.scores_ratio

In [4]:

# restrict the number of free parameters to 3, to make the grid search computationally tractable.
MODEL_PARAMS['ADEXP']
grid_results,sorted_pop = OM.run_simple_grid(2,free_params=['b','C','peak_v'])

['C', 'b', 'peak_v', 'ADAPTATION_TIME_CONSTANT_tau_w', 'ADAPTATION_VOLTAGE_COUPLING_a', 'FIRING_THRESHOLD_v_spike', 'MEMBRANE_RESISTANCE_R', 'MEMBRANE_TIME_SCALE_tau_m', 'RHEOBASE_THRESHOLD_v_rh', 'SHARPNESS_delta_T', 'SPIKE_TRIGGERED_ADAPTATION_INCREMENT_b', 'V_RESET', 'V_REST']
['C', 'b', 'peak_v', 'ADAPTATION_TIME_CONSTANT_tau_w', 'ADAPTATION_VOLTAGE_COUPLING_a', 'FIRING_THRESHOLD_v_spike', 'MEMBRANE_RESISTANCE_R', 'MEMBRANE_TIME_SCALE_tau_m', 'RHEOBASE_THRESHOLD_v_rh', 'SHARPNESS_delta_T', 'SPIKE_TRIGGERED_ADAPTATION_INCREMENT_b', 'V_RESET', 'V_REST']
['C', 'b', 'peak_v', 'ADAPTATION_TIME_CONSTANT_tau_w', 'ADAPTATION_VOLTAGE_COUPLING_a', 'FIRING_THRESHOLD_v_spike', 'MEMBRANE_RESISTANCE_R', 'MEMBRANE_TIME_SCALE_tau_m', 'RHEOBASE_THRESHOLD_v_rh', 'SHARPNESS_delta_T', 'SPIKE_TRIGGERED_ADAPTATION_INCREMENT_b', 'V_RESET', 'V_REST']
['C', 'b', 'peak_v', 'ADAPTATION_TIME_CONSTANT_tau_w', 'ADAPTATION_VOLTAGE_COUPLING_a', 'FIRING_THRESHOLD_v_spike', 'MEMBRANE_RESISTANCE_R', 'MEMBRANE_TIME_S

Z = 0.31
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.07) * V, 'std': array(-1.05) * V}
True
True
True
True
True
{'amplitude': array(84.10058594) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(84.10058594) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(84.10058594) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(84.10058594) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
True


In [23]:
opt=ga_results[0]['pf'][0].dtc.scores_ratio

0.5393517267656796

In [5]:

print('its {0}, that ga beat grid'.format(opt<grid_results[0].dtc.scores))
grid_results[-1].dtc.scores
len(grid_results)
print(grid_results[-1].dtc.scores_ratio)
print(grid_results[0].dtc.scores_ratio)


0.5610156798159149
0.6043685853935559


In [6]:

%%capture
OM = OptMan(local_tests,backend=str('RAW'),boundary_dict=MODEL_PARAMS['RAW'],protocol={'elephant':True,'allen':False,'dm':False})#'tsr':spk_range})
# restrict the number of free parameters to 3, to make the grid search computationally tractable.
grid_results,sp = OM.run_simple_grid(5,free_params=['a','b','C'])
ga_results_izh = OM.optimize(free_params=['b','C','peak_v'],MU=3,NGEN=3)

len([ (gr.dtc.scores,gr.dtc.attrs) for gr in grid_results ])

['a', 'b', 'c']
['a', 'b', 'c']
['a', 'b', 'c']
['a', 'b', 'c']
['a', 'b', 'c']
['a', 'b', 'c']
['a', 'b', 'c']
['a', 'b', 'c']
[<neuronunit.optimisation.data_transport_container.DataTC object at 0x1c2ad51da0>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x1c2ad514a8>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x1c2ad513c8>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x1c2ad51128>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x1c2ad51c88>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x1c2ad51470>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x1c2ad51828>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x1c2ad51d30>]
True
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'mean': arra

True
True
True
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
{'amplitude': array(0.10118821) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.10118821) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.10118821) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.10118821) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
True
True
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 25

True
True
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
{'amplitude': array(0.11690828) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.11690828) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.11690828) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.11690828) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
True
True
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, '

True
True
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
{'amplitude': array(0.07846334) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.07846334) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.07846334) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.07846334) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
True
True
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, '

True
True
True
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
{'amplitude': array(0.11438656) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.11438656) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.11438656) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.11438656) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
True
['a', 'b', 'c']
['a', 'b', 'c']
['a', 'b', 'c']
['a', 'b', 'c']
['a', 'b', 'c']
['a', 'b', 'c']
['a', 'b', 'c']
['a', 'b', 'c']
[<neuronunit.optimisation.data_transport_container.DataTC object at 0x1c36b2aac8>, <neuronunit.optimisation.

...

True
['a', 'b', 'c']
['a', 'b', 'c']
['a', 'b', 'c']
['a', 'b', 'c']
['a', 'b', 'c']
['a', 'b', 'c']
['a', 'b', 'c']
['a', 'b', 'c']
[<neuronunit.optimisation.data_transport_container.DataTC object at 0x1c36a17240>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x1c36a17dd8>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x1c36a179b0>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x1c36a17f98>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x1c36a174a8>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x1c36a17eb8>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x1c36a17588>, <neuronunit.optimisation.data_transport_container.DataTC object at 0x1c36a17390>]
True
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'mean':

True
True
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
{'amplitude': array(0.14348932) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.14348932) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.14348932) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.14348932) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
True
True
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, '

True
True
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
{'amplitude': array(0.10681187) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.10681187) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.10681187) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.10681187) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
True
True
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, '

True
True
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(-0.9783928) * V}
True
True
True
True
True
{'amplitude': array(0.13711707) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.13711707) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.13711707) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
{'amplitude': array(0.13711707) * pA, 'delay': array(100.) * ms, 'duration': array(1000.) * ms}
True
True
True
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, 'std': array(28.74658531) * mV} {'mean': array(-0.06522619) * V, 'std': array(1.) * V}
Z = 0.48
{'value': array(-79.) * mV, 'mean': array(-79.) * mV, 'n': 2578, '

125

In [7]:
[ (gr.dtc.scores,gr.dtc.attrs,gr) for gr in grid_results ] 

[({'RheobaseTest': 8.47477458554291e-05,
   'RestingPotentialTest': 0.36816528919597813,
   'InjectedCurrentAPThresholdTest': 0.9797604836255633,
   'TimeConstantTest': 0.7028267435086151,
   'InjectedCurrentAPAmplitudeTest': 0.9980353270758306,
   'CapacitanceTest': 0.265108402667218,
   'InjectedCurrentAPWidthTest': 0.47951519668537346,
   'InputResistanceTest': 1.0},
  {'a': 0.01, 'b': -2.0, 'c': -60.0},
  [0.01, -2.0, -60.0]),
 ({'RheobaseTest': 8.47477458554291e-05,
   'RestingPotentialTest': 0.36816528919597813,
   'InjectedCurrentAPThresholdTest': 0.9797604836255633,
   'TimeConstantTest': 0.7028267435086151,
   'InjectedCurrentAPAmplitudeTest': 0.9980353270758306,
   'CapacitanceTest': 0.265108402667218,
   'InjectedCurrentAPWidthTest': 0.47951519668537346,
   'InputResistanceTest': 1.0},
  {'a': 0.01, 'b': -2.0, 'c': -55.0},
  [0.01, -2.0, -55.0]),
 ({'RheobaseTest': 8.47477458554291e-05,
   'RestingPotentialTest': 0.36816528919597813,
   'InjectedCurrentAPThresholdTest': 0.97

# Sort this list to find optimal solution from grid search.
* Use optimal solution to seed a genetic algorithm.

This will be a fast way of showing that the genetic algorithm can find a better solution. Alternatively we could start the GA from scratch, but if the genetic algorithm was not working at all, this would be a faster way to demonstrate it. Since time is short, let us do it the fast way first.



In [8]:
dtc_pop = sorted([ (gr.dtc.scores_ratio,gr.dtc.attrs,gr) for gr in grid_results ], key=lambda tup: tup[0])

# Ground truth via exhaustive search is:


In [9]:
print('The model implied by parameters: {0}'.format(dtc_pop[0][1]))
print('Yields the scores: {0}'.format(dtc_pop[0][2].dtc.scores))
print('Yields the ratio scores: {0}'.format(dtc_pop[0][2].dtc.scores_ratio))

The model implied by parameters: {'a': 0.024999999999999998, 'b': 2.25, 'c': -40.0}
Yields the scores: {'RheobaseTest': 0.00014411474714126626, 'RestingPotentialTest': 0.36816528919597813, 'InjectedCurrentAPThresholdTest': 0.9797521225037225, 'TimeConstantTest': 0.703268325083999, 'InjectedCurrentAPAmplitudeTest': 0.9980353270758306, 'CapacitanceTest': 0.2651321421680255, 'InjectedCurrentAPWidthTest': 0.0003164933507386891, 'InputResistanceTest': 1.0}
Yields the ratio scores: 0.5393517267656796


* Now lets create a seed population from the top 10 exhaustive search results.


In [10]:
gene_pop = [d[2] for d in dtc_pop[0:10]]
len(dtc_pop)

125

In [11]:
%%capture 
results = local_tests.optimize(MODEL_PARAMS['RAW'],backend='RAW',protocol={'allen': False, 'elephant': True},\
                               NGEN=len(gene_pop)+len(gene_pop),MU=len(gene_pop)+len(gene_pop), free_params=dtc_pop[0][1].keys(), \
                               seed_pop=gene_pop)
#(NGEN = 10, MU=len(gene_pop), seed_pop = gene_pop,backend='RAW',)
#[ (gr.dtc.scores,gr.dtc.attrs) for gr in grid_results ] 

# Optimization management classes: 
* can also do things like round-trip testing (test the simulators internal performance, as if to bench mark itself with simulated neural data).

In [12]:
results[0]['dtc_pop'][0].scores
opt_out = results[0]['dtc_pop'][0].scores_ratio


In [25]:
grid_out = dtc_pop[0][2].dtc.scores_ratio

print('its true that ga beat exhaustive search {0}'.format(opt_out<grid_out))
print('its true that ga was no worse than exhaustive search {0}'.format(opt_out==grid_out))

print(results[0]['dtc_pop'][0].attrs,'grid best parameters')
print(dtc_pop[0][2].dtc.attrs,'grid best parameters')
grid_out

False
{'a': 0.024999999999999998, 'b': 2.25, 'c': -40.0} grid best parameters
{'a': 0.024999999999999998, 'b': 2.25, 'c': -40.0} grid best parameters


0.5393517267656796

In [24]:
grid_out = dtc_pop[0][2].dtc.scores_ratio
opt_out

0.5393517267656796

In [ ]:
results_no_seed = local_tests.optimize(MODEL_PARAMS['RAW'],backend='RAW',protocol={'allen': False, 'elephant': True},\
                               NGEN=len(gene_pop)+len(gene_pop),MU=len(gene_pop)+len(gene_pop),\
                               free_params=dtc_pop[0][1].keys())


In [ ]:
grid_results_4 = OM.run_simple_grid(4,free_params=['a','b','c','C'])
results_no_seed_4 = local_tests.optimize(MODEL_PARAMS['RAW'],backend='RAW',protocol={'allen': False, 'elephant': True},\
                               NGEN=len(gene_pop)+len(gene_pop),MU=len(gene_pop)+len(gene_pop),\
                               free_params=dtc_pop[0][1].keys())

print('The model implied by parameters: {0}'.format(dtc_pop[0][1]))
print('Yields the scores: {0}'.format(dtc_pop[0][2].dtc.scores))
print('Yields the ratio scores: {0}'.format(dtc_pop[0][2].dtc.scores_ratio))


In [ ]:
%%capture
